In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.


*In Part 1 I will be exploring columns separately and then possibly look into any correlation that
may exist between them.
* Columns will be renamed, checked for null values,datatypes etc...
* New possible columns will be created
* different bean types and brends will be observed

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
choco=pd.read_csv('/kaggle/input/chocolate-bar-ratings/flavors_of_cacao.csv')

In [ ]:
choco.tail()

I am interested to look into 3 columns for begining: bean_region,bean_country and bean_type.In order to make a table a bit more transparent and the order more logical I will also regroup columns

In [ ]:
choco['Specific Bean Origin\nor Bar Name'].unique()

In [ ]:
choco['Bean\nType'].unique()

In [ ]:
choco['Broad Bean\nOrigin'].unique()

In [ ]:
choco.columns

In [ ]:
new_columns=['company','bean_region','review_#','review_date','cocoa_%','company_loc',
             'rating','bean_type','bean_country']
old_columns=choco.columns

In [ ]:
choco=choco.rename(columns=dict(zip(old_columns,new_columns)))

In [ ]:
#renaming columns into more "logical"
choco=choco[['bean_region','bean_country','bean_type','cocoa_%',
       'review_#','review_date','company','company_loc','rating']]

In [ ]:
choco.info()

In [ ]:
choco.head(50)

In [ ]:
choco.groupby('bean_region')[['rating']].mean().sort_values(by='rating',ascending=False).head(30)

In [ ]:
choco[choco['bean_type'].isnull()]

In [ ]:
choco[choco['bean_type']=='Trinitario']

In [ ]:
choco['bean_type'].describe()

It seems that there are alot of null values but they have specific format. We need to discover what it is and try to replace it with NaN

In [ ]:
choco.bean_type.values[1]

In [ ]:
choco['bean_type'].unique()

In [ ]:
sns.set_style('whitegrid')
plt.figure(figsize=(17,6))
sns.countplot(x=choco['bean_type'])
plt.xticks(rotation='vertical')
plt.tight_layout

In [ ]:
choco[choco['bean_type']=='\xa0']

Finally, I will replace these empty entries with 'none' values

In [ ]:
choco=choco.replace('\xa0','none')

In [ ]:
type(choco.bean_type.values[1])

In [ ]:
choco.head(10)

In [ ]:
len(choco[choco['bean_type']=='none'])

In [ ]:
choco['bean_type'].isnull().value_counts()

As we can see one more is left as null value. I will change that one into a string so the entire column has the same data type 

In [ ]:
choco['bean_type'].fillna(value='none',inplace=True)

In [ ]:
choco['bean_type'].isnull().value_counts()

In [ ]:
choco[choco['bean_type']=='none'].count()

As we can see 49.5% have no bean_type,so this columns shouldn't be the one that will affect the rating a  lot. Still we can investigate some other most common bean types

I will count the number of blends or original sorts of Forastero, Trinitario and Criollo

In [ ]:
def find_word(text):
    if 'Forastero'in text:
        return 1
    else:
        return 0

In [ ]:
choco['bean_type'].apply(find_word).sum()

As we see Forestero appears 201 times as originalsort or part of a blend.I will plot this

In [ ]:
choco['Forastero']=choco['bean_type'].apply(find_word)

In [ ]:
choco['Forastero']

In [ ]:
sns.countplot(x='Forastero',data=choco)

In [ ]:
def find_word(text):
    if 'Trinitario'in text:
        return True
    else:
        return False

In [ ]:
choco['bean_type'].apply(find_word).value_counts()

As we see Forestero appears 476 times as original sort or part of a blend.I will plot this

In [ ]:
choco['Trinitario']=choco['bean_type'].apply(find_word)

In [ ]:
sns.countplot(x='Trinitario',data=choco)

In [ ]:
def find_word(text):
    if 'Criollo'in text:
        return 1
    else:
        return 0

In [ ]:
choco['Criollo']=choco['bean_type'].apply(find_word)

In [ ]:
choco['Criollo'].sum()

As we see Forestero appears 225 times as original sort or part of a blend.I will plot this

In [ ]:
sns.countplot(x='Criollo',data=choco)

What about the number of original sorts ? Can we count those and can we count blends?

In [ ]:
def is_blend(text):
    if (',' in text) or('Blend' in text) or ('mix'in text) :
        return 1
    else:
        return 0


In [ ]:
is_blend('Amazon Blend')

In [ ]:
choco['blend']=choco['bean_type'].apply(is_blend)

In [ ]:
choco['blend'].value_counts()

So we have 103 blend types.We have to be carefull here. We know that we have 888 null values, so that means that out of 1692 'non blends' 888 of them are null values. So we have actually 804 non-blends. We need to make a column that will represent this

In [ ]:
choco1=pd.concat((choco['Forastero'],choco['Trinitario'],choco['Criollo'],choco['blend']),axis=1)

In [ ]:
choco1

In [ ]:
fig, ax =plt.subplots(1,4)
sns.countplot(choco1['Forastero'], ax=ax[0])
sns.countplot(choco1['Trinitario'], ax=ax[1])
sns.countplot(choco1['Criollo'], ax=ax[2])
sns.countplot(choco1['blend'], ax=ax[3])
plt.tight_layout()

In [ ]:
choco.groupby('blend')['rating'].mean()

In [ ]:
choco.groupby('Forastero')['rating'].mean()

In [ ]:
choco.groupby('Criollo')['rating'].mean()

In [ ]:
choco.groupby('Trinitario')['rating'].mean()

Conclusion: It seems that 3 above mentioned are the most common bean types. But when we look at the rating of these 3 , the avearge we are seeing is not significant as these bean types are also counted in blends, so therefor these avearges are not showing avearge for chocolates with ONLY that bean type. We need to check how many of these are actually only Criollo,Forastero or...

In [ ]:
def not_blend(text):
    if ('Forastero'in text) and (',' not in text) and ('Blend' not in text) and  ('mix'not in text) :
        return 1
    else:
        return 

In [ ]:
choco['bean_type'].apply(not_blend).sum()

So we can see that there is a difference. Before we had 201 identified 'Forastero' types, but only 194 of them are only Forastero! So 7 of them belong to blends. I will do this with other 2 bean types. I could define pnly one function and change paramter.

In [ ]:
def not_blend(name, text):
    if (name in text) and (',' not in text) and ('Blend' not in text) and  ('mix'not in text) :
        return 1
    else:
        return 0
choco['Forastero']=choco['bean_type'].apply(lambda text: not_blend("Forastero",text))
choco['Criollo']=choco['bean_type'].apply(lambda text: not_blend("Criollo",text))
choco['Trinitario']=choco['bean_type'].apply(lambda text: not_blend("Trinitario",text))

In [ ]:
choco['Forastero'].sum()

In [ ]:
choco['Criollo'].sum()

In [ ]:
choco['Trinitario'].sum()

In [ ]:
choco.groupby("Forastero")['rating'].mean()

In [ ]:
choco.groupby("Trinitario")['rating'].mean()

In [ ]:
choco.groupby("Criollo")['rating'].mean()

In [ ]:
ratings=pd.DataFrame(data=[choco.groupby("Forastero")['rating'].mean(),
                          choco.groupby("Trinitario")['rating'].mean(),
                          choco.groupby("Criollo")['rating'].mean(),
                          choco.groupby("blend")['rating'].mean()], index=["Forastero",'Trinitario',
                                                                             'Criollo','blend'], 
                                                                     columns=None)

In [ ]:
ratings

As we can see, the blend has the highest average rating followed by 'Criollo' bean type with average score 3.26. The least popular seems to be Forastero, with the score of 3.11. The blend is still apparentyl the most popular "solution".

PART 2: In ths part I will investigate bean origin vs rating and percentage of cocoa vs rating. 

In [ ]:
choco.head()

In [ ]:
choco['bean_country'].nunique()

In [ ]:
choco['bean_country'].unique()

As we can see some of the beans/blends have omore tha one countrylisted as an origin. I will look into the top 20 countries and the average rating of their beans. I will not count in the appearnce of countries along with some other ones as the avearge rating in that case is skewed by the presence of different countries. Therefore, the average rating for single country would not present the rating for country ONLY.

In [ ]:
choco[choco['bean_country'].isnull()]

In [ ]:
choco['bean_country'].fillna(value='Madagascar',inplace=True)

In [ ]:
choco[choco['bean_country'].isnull()]

In [ ]:
choco[choco['bean_country']=='none'][['bean_region','bean_type']].head(40)

As we can see from above it is hard to identify cocoa beans with only one country when country is missing. For example Trinitario may be from Venezuela, Columbia or Caribbean. So I will ignore those entries where bean_country has "none" value. That would be 73 entries out of 1795 and it accounts to 4% of total data which is acceptable.

In [ ]:
len(choco[choco['bean_country']=='none'])

In [ ]:
choco2=choco.drop(choco[choco['bean_country']=='none'].index)

In [ ]:
len(choco2[choco2['bean_country']=='none'])

In [ ]:
choco2.apply(len)

In [ ]:
country_count=pd.DataFrame(choco2['bean_country'].value_counts().head(24),
                           index=None,columns=None)

In [ ]:
country_count

In [ ]:
country_count.columns

In [ ]:
country_count.index

In [ ]:
country_count.sum()/17.22

In [ ]:
country_count.head(5).sum()/17.22

From what we can see he top 5 countries are origin for almost 50% of data. 
Around 88% of total data come from the above mentioned 24 countries.
Only 12%  or 205 data comes from the rest 75 countries. 
It seems that those 24 countries are the reasonable choice for analysis. I will presetn those on the barchart.

In [ ]:
country_count.plot(kind='bar',color='green')

In [ ]:
l=[]
for country in country_count.index:
    l.append(round(choco[choco['bean_country']==country]['rating'].mean(),2))
    
print(f'{l}')

In [ ]:
l

For the simplicity and teh make sure I can plot them using pltly I will create ISO-3 codes for those countries.

In [ ]:
import pycountry

input_countries = country_count.index

countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_3

codes = [countries.get(country, 'Unknown code') for country in input_countries]

print(codes)

In [ ]:
#replacing missing values
Country_code=['VEN', 'ECU', 'PER', 'MDG', 'DOM', 'NIC', 'BRA', 'BOL', 'BLZ', 'PNG', 'COL', 'CRI',
              'VNM', 'TZA', 'TTO', 'GHA', 'MEX', 'US-HI', 'GTM', 'DOM', 'JAM', 'GRD', 'IDN', 'HND']

In [ ]:
top_24_rating=pd.DataFrame(list(zip(Country_code,l)),country_count.index,columns=['Country Code','Rating'])

In [ ]:
top_24_rating

In [ ]:
top_24_countries=pd.concat([country_count,top_24_rating],axis=1)

In [ ]:
top_24_countries=top_24_countries[['Country Code','bean_country','Rating']]

In [ ]:
top_24_countries.rename(columns={'bean_country':'# of data'},inplace=True)

In [ ]:
top_24_countries

In [ ]:

from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

In [ ]:
data=dict(type='choropleth',
         locations=Country_code, 
         z=top_24_countries['Rating'],
         text=top_24_countries['Country Code'],
         colorbar={'title':'Chocolate rating'})
layout=dict(title='Chocolate rating for top 24 beans origin countries',
           geo=dict(showframe=False,projection={'type':'mercator'}))

choromap4=go.Figure(data=[data],layout=layout)

iplot(choromap4)

In [ ]:
#countplot per countries with associated mean score
def display_figures(ax):
    i=0
    for p in ax.patches:
        h=p.get_height()
        if (h>0):
            value=l[i]
            ax.text(p.get_x()+p.get_width()/2,h+6, value,ha='center')
            i=i+1
            
            
plt.figure(figsize=(15,10))
ax=sns.barplot(x='Country Code',y='# of data',data=top_24_countries,orient='v')
display_figures(ax)

In [ ]:
#barplot for mean score vs.countries
def display_figures(ax):
    i=0
    for p in ax.patches:
        h=p.get_height()
        if (h>0):
            value=top_24_countries['# of data'][i]
            ax.text(p.get_x()+p.get_width()/2,h+0.08, value, ha='center')
            i=i+1
            
            
plt.figure(figsize=(15,10))
ax=sns.barplot(x='Country Code',y='Rating',data=top_24_countries,orient='v')
display_figures(ax)

In [ ]:
top_24_countries['Rating'].max()

In [ ]:
top_24_countries['Rating'].min()

As we can notice thre are no huge differences in average average rating amongst the top 24 countries. All average ratings are withing the range (3,3.35). As fort he average rating there are not hge differences between thesetop 24countries, so 88%of data has rating within the range (3,3.35).

It is important to see the top ratings, number of ratings and their origin. Let's look at 
ratings ≥ 3.5, number of reviews associated, bean_type and origin.

In [ ]:
choco.groupby('bean_country')[['rating']].mean().sort_values(by='rating',ascending=False).head(20)

In [ ]:
#making a new dataframe for countries with the highest reviews
df1=pd.DataFrame(round(choco.groupby('bean_country')[['rating']].mean().sort_values(by='rating',ascending=False).head(20),2),
                           index=None,columns=None)

In [ ]:
df1

In [ ]:
l1=[]
for country in df1.index:
    l1.append(len(choco[choco['bean_country']==country]))
    

In [ ]:
len(l1)

In [ ]:
#adding the number of reviews to top 20 ratings
top_20_ratings=pd.DataFrame(list(zip(l1,df1['rating'])),index=df1.index,columns=['# of reviews','Avearge rating'])

In [ ]:
top_20_ratings

In [ ]:
top_20_ratings['# of reviews'].sum()/17.22

From here we can see that only 2 of 20 top ratings come from 2 or  3 reviews and the rest are all coming from 1 review. Therefore, we have no enough evdiences to say that those are the top rated chocolates indeed. 

Also, the top 20 reviews account only to 1.3% of data which means that the probability of being top 20 is 0.01

Let'slook at the ratings below 3. 

In [ ]:
df2=pd.DataFrame(round(choco.groupby('bean_country')[['rating']].mean().sort_values(by='rating',ascending=True).head(20),2),
                           index=None,columns=None)

In [ ]:
df2

In [ ]:
l2=[]
for country in df2.index:
    l2.append(len(choco[choco['bean_country']==country]))

In [ ]:
bottom_20_ratings=pd.DataFrame(list(zip(l2,df2['rating'])),index=df2.index,columns=['# of reviews','Avearge rating'])

In [ ]:
bottom_20_ratings.tail()

This gives us a bit better picture. We can see that beans from Caribbean,Uganda.St.Lucia,Ivory Coast,West africa seem not be very popular.Let's exmine them more closely.

In [ ]:
choco[choco['bean_country']=='Carribean']

In [ ]:
choco[choco['bean_country']=='Uganda']

In [ ]:
choco[choco['bean_country']=='Ivory Coast']

In [ ]:
choco[choco['bean_country']=='West Africa']

In [ ]:
(bottom_20_ratings['# of reviews'].sum()-73)/17.22

From this we can see that Uganda has the highest percentage of reviews less than 3. Still the bottom 20 countries that have reviews less than 3, if we don't count hte missing data,account to only 3% of total data, which is not very significant. It means that the probablilty of review being less than 3 is only 0.03. 

This also tells us that the majority of reviews are "in the middle", which is anyway characteristic for normal distribution.

In [ ]:
sns.distplot(choco['rating'],kde=False,bins=8)

We can see from here that data are slightly negatively skewed, but they are still approximately normally distributed. 




PART 3: In this part I will investigate percentage of cocoao and the average rating. Also, it is well know that for example  Italy has a 'good" chocolates, so I will also examine the company location and the rating.

In [ ]:
#checking on company location
choco['company_loc'].nunique()

This is great- there are only 60 company locations. Let's check for null data.

In [ ]:
choco['company_loc'].isnull().value_counts()

In [ ]:
choco['company_loc'].unique()

In [ ]:
choco.groupby('company_loc')[['rating']].mean()

In [ ]:
choco=choco.replace('Amsterdam','Netherlands')

In [ ]:
choco.groupby('company_loc')[['rating']].mean()

In [ ]:
choco['company_loc'].value_counts()

In [ ]:
df3=pd.DataFrame(choco['company_loc'].value_counts().head(15),index=None,columns=None)

I will observe oly top 15 companies because the ones after account to <1% of total reviews. 

In [ ]:
df3

In [ ]:
l3=[]
for country in df3.index:
    l3.append(round(choco[choco['company_loc']==country]['rating'].mean(),2))



In [ ]:
l3

In [ ]:
top_companies=pd.DataFrame(list(zip(df3['company_loc'],l3)),index=df3.index,columns=['# of reviews','Rating'])

In [ ]:
top_companies.sort_values(by='Rating',ascending=False)

So we can seethat the best average rating goes to Australia,Switzerland,Italy,Canada and Spain. 

All top 15 countries again have reviews from 3.01 to 3.36.

In [ ]:
top_24_countries

In [ ]:
s=0
for country in  top_companies.index:
    if country in top_24_countries.index:
        s+=1
    else:
        pass
print (s)


From this we can see that out of top 24 beans' origin countries, only 3 of them are actualy having company locations amongst top 15. This tells us that most of the better quality beans are exported and chocolate is produced somewhere else.  

In [ ]:
choco['cocoa_%']

In [ ]:
def per_to_num(data):
    return int(data[0:2])

In [ ]:
choco['cocoa_%']=choco['cocoa_%'].apply(per_to_num)

In [ ]:
choco['cocoa_%'].unique()

In [ ]:
choco['cocoa_%'].value_counts().head(10).sum()/17.95

So, 81 percent of data has cocoa % in thetop 10 percentages. I will create the dataset representing those percentages and look at the avearge ratings.

In [ ]:
# %top 10 cocoa percentages
df4=pd.DataFrame(choco['cocoa_%'].value_counts().head(10),index=None,columns=None)

In [ ]:
df4

In [ ]:
df5=choco.groupby('cocoa_%')[['rating']].mean()

In [ ]:
df5

In [ ]:
plt.scatter(x=df5.index,y='rating',data=df5)

This scatter plot shows us that majority of chocolates with the rating between 3 and 3.5 have cocoa percentage somwhere between 60% and 80%

In [ ]:
#let'sgo backand look at the top 10 cocoa percentages
l4=[]
for per in df4.index:
    l4.append(round(choco[choco['cocoa_%']==per]['rating'].mean(),2))



In [ ]:
l4

In [ ]:
top_10_cocoa_per=pd.DataFrame(list(zip(df4['cocoa_%'],l4)),index=df4.index,columns=['# of reviews','Rating'])

In [ ]:
top_10_cocoa_per

In [ ]:
plt.scatter(x=top_10_cocoa_per.index,y='Rating',s=top_10_cocoa_per['# of reviews'], data=top_10_cocoa_per,marker='o',c='pink')

As we can see the most popular chocolates are those with the rating 70% and it has the second highest review. 

Now looking back at those "unpopoular countries".

In [ ]:
choco[choco['bean_country']=='Carribean']['cocoa_%'].mean()

Average rating for 76% cocoa is 2.9 , so it could be the cocoa % they are using.

In [ ]:
choco[choco['bean_country']=='Uganda']['cocoa_%'].mean()

In [ ]:
choco[choco['bean_country']=='Ivory Coast']['cocoa_%'].mean()

In [ ]:
choco[choco['bean_country']=='West Africa']['cocoa_%'].mean()

In [ ]:
df5

In [ ]:
plt.figure(figsize=(15,6))
sns.boxplot(x='cocoa_%', y='rating',data=choco)